In [39]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import time
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline  
from scipy import stats

from collections import Counter
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
import re
import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [40]:
data= pd.read_csv(r'C:\Users\injoo\광주 인공지능 사관학교\로테이션\자연어처리&추천시스템 김준태\미니프로젝트\Dataset\cleaned_USvidoes.csv', error_bad_lines=False)
data = data[['video_id', 'views', 'date']]

In [41]:
data2= pd.read_csv(r'C:\Users\injoo\광주 인공지능 사관학교\로테이션\자연어처리&추천시스템 김준태\미니프로젝트\Dataset\USvideos.csv', error_bad_lines=False)
data2 = data2['title']

b'Skipping line 2401: expected 11 fields, saw 21\nSkipping line 2800: expected 11 fields, saw 21\nSkipping line 5297: expected 11 fields, saw 12\nSkipping line 5299: expected 11 fields, saw 12\nSkipping line 5300: expected 11 fields, saw 12\nSkipping line 5301: expected 11 fields, saw 12\n'


In [42]:
df = pd.concat([ data2, data], axis = 1)

In [43]:
us_videos = df

In [44]:
result = us_videos.drop_duplicates('video_id', keep='last').values
df = pd.DataFrame(result, columns = us_videos.columns)

In [45]:
df

,title,video_id,views,date
0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,5ywKal6-anc,703750,2017-09-13
1,iPhone X — Introducing iPhone X — Apple,GGm0FQ6i74U,244433,2017-09-13
2,My Response,qJJHhVf3_ZM,220661,2017-09-13
3,Apple iPhone X first look,9CILSvf1snQ,27794,2017-09-13
4,iPhone X (parody),k9bYO7uSnrg,14577,2017-09-13
...,...,...,...,...
2360,Wonderstruck Official Trailer [HD] | Amazon St...,qRoVlH1OcI4,55762,2017-01-22
2361,Hidden NES Golf game on Nintendo Switch. A tri...,EoejGgUNmVU,142908,2017-01-22
2362,WATCH: First Lady Melania Trump discusses chil...,MT1CMTI0EVw,24532,2017-01-22
2363,First footage of wild sand cat kittens,AFxLA3RGjnc,144039,2017-01-22


In [46]:
df.isnull().sum()

title       0
video_id    1
views       1
date        1
dtype: int64

In [47]:
df= df.dropna()

In [48]:
df = df.drop(columns = ['video_id', 'date'])

In [49]:
views = df['views']

In [50]:
views.quantile([0.25, 0.5, 0.75, 1])

0.25        83536.5
0.50         287788
0.75         920616
1.00    4.15007e+07
Name: views, dtype: object

In [51]:
df['views_label'] = df['views']

In [52]:
def label_func(x):
    if x>=287788:
        return 1
    else:
        return 0

In [53]:
df['views_label'] = df['views_label'].apply(lambda x: label_func(x))

In [54]:
df

,title,views,views_label
0,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,703750,1
1,iPhone X — Introducing iPhone X — Apple,244433,0
2,My Response,220661,0
3,Apple iPhone X first look,27794,0
4,iPhone X (parody),14577,0
...,...,...,...
2359,OUR TEAM 10 AUDITION with TESSA BROOKS & NICK ...,440393,1
2360,Wonderstruck Official Trailer [HD] | Amazon St...,55762,0
2361,Hidden NES Golf game on Nintendo Switch. A tri...,142908,0
2362,WATCH: First Lady Melania Trump discusses chil...,24532,0


In [55]:
df.describe()

,views_label
count,2364.000000
mean,0.500000
std,0.500106
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [56]:
# df = df.drop(columns=['views'])
title_data = df

# 이제 자연어 처리 해주자 제목에 대해서

In [83]:
df

,title,views,views_label
0,year vlogging logan paul changed youtube forever,703750,1
1,iphone x introducing iphone x apple,244433,0
2,response,220661,0
3,apple iphone x first look,27794,0
4,iphone x parody,14577,0
...,...,...,...
2359,team audition tessa brooks nick crompton,440393,1
2360,wonderstruck official trailer hd amazon studios,55762,0
2361,hidden nes golf game nintendo switch tribute s...,142908,0
2362,watch first lady melania trump discusses child...,24532,0


In [84]:
df['title'] = df['title'].str.replace("[^\w]", " ")

In [85]:
df['title'] = df['title'].replace('', np.nan)

In [86]:
title_data = df['title']

In [87]:
titles_list = []
#숫자 제거가 덜 되었다. 하나하나 돌면서 숫자도 제거해주자. 단 단어랑 붙어 있는 숫자는 살려주자
for title in title_data:
    title = title.split(" ")
    word_without_number = []
    for word in title:
        output = re.sub(r'\d+', '', word)
        if output != '':
            word_without_number.append(output.lower())
    titles_list.append(word_without_number)

In [88]:
titles_list

[['year', 'vlogging', 'logan', 'paul', 'changed', 'youtube', 'forever'],
 ['iphone', 'x', 'introducing', 'iphone', 'x', 'apple'],
 ['response'],
 ['apple', 'iphone', 'x', 'first', 'look'],
 ['iphone', 'x', 'parody'],
 ['disaster', 'artist', 'official', 'trailer', 'hd'],
 ['check', 'hud', 'ben', 'carson', 'hurricanes'],
 ['iphone', 'x', 'impressions', 'hands'],
 ['attacked', 'police', 'dog'],
 ['honest', 'trailers', 'mummy'],
 ['honest', 'college', 'tour'],
 ['best', 'floyd', 'mayweather', 'interview', 'awkward', 'puppets'],
 ['jennifer', 'lawrence', 'challenges', 'jimmy', 'axe', 'throwing', 'contest'],
 ['hand', 'hand', 'benefit', 'hurricane', 'relief', 'mtv'],
 ['colin',
  'cloud',
  'mind',
  'reader',
  'predicts',
  'tweets',
  'america',
  'got',
  'talent'],
 ['iphone', 'x', 'hands', 'everything', 'need', 'know'],
 ['want', 'eat'],
 ['getting', 'conversation', 'language', 'actually', 'speak', 'well'],
 ['juicy', 'chicken', 'breast', 'suck', 'cooking', 'episode'],
 ['downsizing', 

In [89]:
# only_title = df['title']
# title_data = only_title

# title_data = title_data.str.replace("[^\w]", " ")
# #혹시나 공백이 있으면
# title_data= title_data.replace('', np.nan)

# #결측치 있으면 모두 제거
# title_data = title_data.dropna(how='any')


### Stopwords 제거 (NLTK corpus stopwords 이용)

In [90]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 
cleaned_titles = []
words_list = []
for title in titles_list:
    tmp_list = []
    for word in title:
        if word not in stop_words:
            tmp_list.append(word)
            words_list.append(word)
    cleaned_titles.append(tmp_list)

In [91]:
x = pd.Series(cleaned_titles) 

In [92]:
df['title'] = x

In [93]:
print(type(df['title'][0]))

<class 'list'>


In [94]:
df

,title,views,views_label
0,"[year, vlogging, logan, paul, changed, youtube...",703750,1
1,"[iphone, x, introducing, iphone, x, apple]",244433,0
2,[response],220661,0
3,"[apple, iphone, x, first, look]",27794,0
4,"[iphone, x, parody]",14577,0
...,...,...,...
2359,"[team, audition, tessa, brooks, nick, crompton]",440393,1
2360,"[wonderstruck, official, trailer, hd, amazon, ...",55762,0
2361,"[hidden, nes, golf, game, nintendo, switch, tr...",142908,0
2362,"[watch, first, lady, melania, trump, discusses...",24532,0


In [95]:
def strip_func(x):
    return ' '.join(x)

In [96]:
df['title'] = df['title'].apply(lambda x: strip_func(x))

In [97]:
df

,title,views,views_label
0,year vlogging logan paul changed youtube forever,703750,1
1,iphone x introducing iphone x apple,244433,0
2,response,220661,0
3,apple iphone x first look,27794,0
4,iphone x parody,14577,0
...,...,...,...
2359,team audition tessa brooks nick crompton,440393,1
2360,wonderstruck official trailer hd amazon studios,55762,0
2361,hidden nes golf game nintendo switch tribute s...,142908,0
2362,watch first lady melania trump discusses child...,24532,0


# LSTM3에서 가져온 코드다

In [98]:
#test/train 스플릿
X_train , X_test, y_train, y_test = train_test_split(df['title'], df['views'], test_size=0.2, shuffle=False)

# title_train, title_test, y_train, y_test = train_test_split(df['title'], df['views'], test_size=0.2, shuffle=False)

print("# split done")
stopwords = []
#토근화 진행
X_train = []
for stc in title_train:
  token = []
  words = stc.split()
  for word in words:
    if word not in stopwords:
      token.append(word)
  X_train.append(token)

X_test =  []

for stc in title_test:
  token = []
  words = stc.split()
  for word in words:
    if word not in stopwords:
      token.append(word)
  X_test.append(token)

print("# tokenization done")

# split done
# tokenization done


In [99]:
X_train

[['year', 'vlogging', 'logan', 'paul', 'changed', 'youtube', 'forever'],
 ['iphone', 'x', 'introducing', 'iphone', 'x', 'apple'],
 ['response'],
 ['apple', 'iphone', 'x', 'first', 'look'],
 ['iphone', 'x', 'parody'],
 ['disaster', 'artist', 'official', 'trailer', 'hd'],
 ['check', 'hud', 'ben', 'carson', 'hurricanes'],
 ['iphone', 'x', 'impressions', 'hands'],
 ['attacked', 'police', 'dog'],
 ['honest', 'trailers', 'mummy'],
 ['honest', 'college', 'tour'],
 ['best', 'floyd', 'mayweather', 'interview', 'awkward', 'puppets'],
 ['jennifer', 'lawrence', 'challenges', 'jimmy', 'axe', 'throwing', 'contest'],
 ['hand', 'hand', 'benefit', 'hurricane', 'relief', 'mtv'],
 ['colin',
  'cloud',
  'mind',
  'reader',
  'predicts',
  'tweets',
  'america',
  'got',
  'talent'],
 ['iphone', 'x', 'hands', 'everything', 'need', 'know'],
 ['want', 'eat'],
 ['getting', 'conversation', 'language', 'actually', 'speak', 'well'],
 ['juicy', 'chicken', 'breast', 'suck', 'cooking', 'episode'],
 ['downsizing', 

In [100]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding, LSTM

# X_train 단어들을 토대로 정수 인덱스 설정
# 빈도수가 높은 것부터 4000개만 정수 인덱스로 변환하겠다

tokenizer = Tokenizer(2155)
tokenizer.fit_on_texts(X_train)

#위에서 설정된 정수 인덱스를 토대로 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print("# int_encoding done")

# int_encoding done


In [101]:
print(len(tokenizer.word_index))

2230


In [102]:
max_length = 0
for data in X_train:
  if max_length < len(data):
    max_length = len(data)

print(max_length)

19


In [103]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 19
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [104]:
model = Sequential()
model.add(Embedding(2155, 10))
model.add(LSTM(10))
model.add(Dense(1, activation = 'relu'))

In [105]:
model.compile(loss="cate_crossentropy", optimizer = 'adam', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [80]:
# model = Sequential()
# model.add(Embedding(5000, 32))
# model.add(LSTM(32))
# model.add(Dense(1, activation='relu'))

# model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

# model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [ ]:
sentence = input()
#토큰화
token_stc = sentence.split()
#정수 인코딩
encode_stc = tokenizer.texts_to_sequences([token_stc])
#패딩
pad_stc = pad_sequences(encode_stc, maxlen=50)

score = model.predict(pad_stc)
print(score)